In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from efficientdet.data_pipeline.utils import xyxy_to_xywh, xywh_to_xyxy
from efficientdet.data_pipeline.anchors import AnchorBox
from efficientdet.data_pipeline.target_encoder import TargetEncoder
from efficientdet.data_pipeline.input_dataset import create_combined_dataset, create_images_dataset, create_bbox_dataset, create_labels_dataset
from efficientdet.model.efficientdet import EfficientDet
from efficientdet.model.losses import HuberRegressionLoss, FocalClassificationLoss

# Create model

In [ ]:
model = EfficientDet(phi=0)
model.build((None, None, None, 1))
# model.summary()

# Load data

In [ ]:
path_train = '../data/train.csv'
path_test = '../data/test.csv'


def read_csv(path):
    df = pd.read_csv(path, dtype={'img_path': str, 'x1': 'int32', 'y1': 'int32', 'x2': 'int32', 'y2': 'int32', 'label': 'int32'})
    print(df.shape)
    return df


df_train = read_csv(path_train)
df_test = read_csv(path_test)

# Test set predictions

In [ ]:
model.load_weights('../data/results/checkpoints/efficientdet-20.hdf5')

In [ ]:
ds_train = create_images_dataset(df_train)
ds_test = create_images_dataset(df_test)

In [ ]:
def add_box(ax, bbox, ec='r', fc='none', lw=0.5, label=None, score=None):
    x1, y1, x2, y2 = bbox
    height = y2 - y1
    width = x2 - x1
    rect = Rectangle((x1, y1), width, height, ec=ec, fc=fc, lw=lw)
    ax.add_patch(rect)
    text = []
    if label is not None:
        text.append("label: {}".format(int(label)))
    if score is not None:
        text.append("score: {:.2f}".format(score))
    if text:
        ax.text(x1, y1, ", ".join(text), fontsize=14, color='white')

In [ ]:
n_cols = 3
n_rows = 5

fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 6 * n_rows))

for ax, img in zip(np.ravel(axes), ds_train.take(n_rows * n_cols)):
    ax.imshow(img)
    img = tf.expand_dims(img, axis=0)
    boxes, scores, classes = model(img)
    for (box, pred_label, score) in zip(boxes[0], classes[0], scores[0]):
        if score > 0:
            add_box(ax, box, label=pred_label.numpy(), score=score)